In [1]:
import pandas as pd
import numpy as np
import json
from scipy.sparse import csr_matrix, identity
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics import accuracy_score, f1_score
import itertools
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA

/Users/manyuhaochi/Desktop/recomtest/venv/lib/python3.11/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


## Load imputed data

In [5]:
df_imputed_original = pd.read_csv('../data/processed/imputed_data_with_label.csv')

## Perform PCA on existing data

In [6]:
target_cols = ['Automotive & Transportation Services', 'Clothing & Fashion',
       'Digital Goods & Computers', 'Electronics & Appliances',
       'Freight & Trucking', 'Hotels & Accommodation',
       'Legal & Financial Services', 'Machinery & Tools',
       'Medical & Healthcare Services', 'Movies & Theaters',
       'Postal Services - Government Only', 'Rail & Bus Transport',
       'Restaurants & Eating Places', 'Retail Stores',
       'Sports & Recreational Activities', 'Steel & Metal Products',
       'Telecommunications & Media', 'Utilities & Home Services']

# List of feature columns used for imputation
feature_cols = ['current_age', 'retirement_age', 'birth_month', 'gender',
       'latitude', 'longitude', 'yearly_income', 'total_debt', 'credit_score',
       'num_credit_cards', 'Credit', 'Debit', 'Debit (Prepaid)']

label_cols = ['Label_Rewards_Credit_Card', 'Label_Insurance_Solutions',
              'Label_Digital_Financing', 'Label_Home_Improvement_Loan',
              'Label_Auto_Vehicle_Financing', 'Label_Commodity_Investment_Services',
              'Label_Travel_Rewards_Card', 'Label_Savings_Investment_Plans',
              'Label_Wealth_Management_Savings']

Y = df_imputed_original[label_cols].copy()
Y_bin = (Y >= 2).astype(int)

# Prepare user features for tuning.
# Define two feature sets:
base_features = feature_cols.copy()
expanded_features = feature_cols + target_cols  # expanded: include spending categories

## Before performing PCA, we want to find the number of components that is able to explain 90% of the variance, eventually we choose n = 22.

In [7]:
X_expanded = df_imputed_original[expanded_features].copy()
scaler_exp = StandardScaler()
X_expanded_scaled = scaler_exp.fit_transform(X_expanded)

# Assume X_expanded_scaled is already computed (using StandardScaler on your expanded features)
n_total = X_expanded_scaled.shape[1]  # should be 40

for n in range(1, n_total + 1):
    pca_temp = PCA(n_components=n, random_state=42)
    pca_temp.fit(X_expanded_scaled)
    cum_explained = np.sum(pca_temp.explained_variance_ratio_)
    print(f"n_components = {n}: Cumulative explained variance = {cum_explained:.4f}")

n_components = 1: Cumulative explained variance = 0.1587
n_components = 2: Cumulative explained variance = 0.2356
n_components = 3: Cumulative explained variance = 0.2952
n_components = 4: Cumulative explained variance = 0.3487
n_components = 5: Cumulative explained variance = 0.3901
n_components = 6: Cumulative explained variance = 0.4300
n_components = 7: Cumulative explained variance = 0.4668
n_components = 8: Cumulative explained variance = 0.5020
n_components = 9: Cumulative explained variance = 0.5360
n_components = 10: Cumulative explained variance = 0.5682
n_components = 11: Cumulative explained variance = 0.6002
n_components = 12: Cumulative explained variance = 0.6311
n_components = 13: Cumulative explained variance = 0.6614
n_components = 14: Cumulative explained variance = 0.6909
n_components = 15: Cumulative explained variance = 0.7190
n_components = 16: Cumulative explained variance = 0.7466
n_components = 17: Cumulative explained variance = 0.7735
n_components = 18: Cumu

In [ ]:
def compute_top3_accuracy_for_fold(model, X_val, interactions_val, item_features, k=3):
    num_users = X_val.shape[0]
    top3_acc = []
    users_with_purchases = 0  # Track users who actually bought something
    
    for user_id in range(num_users):
        # Predict scores for all items for this user
        scores = model.predict(user_id, np.arange(interactions_val.shape[1]),
                             user_features=X_val,
                             item_features=item_features)
        
        # Get top k predicted item indices
        top3_indices = np.argsort(-scores)[:k]
        
        # Get true purchased item indices
        true_positives = set(np.where(interactions_val[user_id].toarray().flatten() == 1)[0])
        n_true = len(true_positives)
        
        # Skip users with no purchases (do not count them in accuracy)
        if n_true == 0:
            continue
        
        users_with_purchases += 1
        
        # Case 1: User purchased 3+ items
        if n_true >= 3:
            count = len(set(top3_indices).intersection(true_positives))
            top3_acc.append(count / 3.0)
        
        # Case 2: User purchased exactly 2 items
        elif n_true == 2:
            intersection = set(top3_indices).intersection(true_positives)
            if len(intersection) == 2:
                top3_acc.append(1.0)
            elif len(intersection) == 1:
                top3_acc.append(0.5)
            else:
                top3_acc.append(0)
        
        # Case 3: User purchased exactly 1 item
        elif n_true == 1:
            if top3_indices[0] in true_positives:
                top3_acc.append(1.0)
            elif len(set(top3_indices[1:]).intersection(true_positives)) > 0:
                top3_acc.append(0.5)
            else:
                top3_acc.append(0)
    
    # Return 0 if no users made purchases (edge case)
    if users_with_purchases == 0:
        return 0.0
    
    return np.mean(top3_acc)

In [ ]:
def grid_search_cv(feature_list, X_train_full, Y_train_bin):
    # Extract the features from X_train_full based on the provided feature list.
    X_train_features = X_train_full[feature_list].copy()
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_features)
    user_features = csr_matrix(X_train_scaled)
    
    # Build the interaction matrix from Y_train_bin.
    interactions = csr_matrix(Y_train_bin.values)
    num_items = interactions.shape[1]
    item_features = identity(num_items, format='csr')
    
    # Define hyperparameter grid.
    param_grid = {
        'loss': ['warp', 'bpr','logistic'],
        'no_components': [16, 32, 64],
        'learning_rate': [0.001, 0.01, 0.05],
        'epochs': [30, 50],
        'user_alpha': [1e-5, 1e-4],
        'item_alpha': [1e-5, 1e-4]
    }
    
    # Set up 5-fold CV.
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    grid_results = []
    upsample_factor = 2

    for loss, no_components, learning_rate, epochs, user_alpha, item_alpha in itertools.product(
        param_grid['loss'],
        param_grid['no_components'],
        param_grid['learning_rate'],
        param_grid['epochs'],
        param_grid['user_alpha'],
        param_grid['item_alpha']
    ):
        fold_top3_acc = []
        fold_prec = []

        for train_idx, val_idx in kf.split(user_features):
            X_train_cv = user_features[train_idx]
            X_val_cv = user_features[val_idx]
            
            # Get the training interactions for this fold.
            fold_train = interactions[train_idx].toarray().astype(float)
            
            # Upsample the minority class for each product column in the training fold.
            for j in range(fold_train.shape[1]):
                pos_count = np.sum(fold_train[:, j] == 1)
                neg_count = np.sum(fold_train[:, j] == 0)
                if pos_count / fold_train.shape[0] < 0.3:
                    fold_train[:, j] = np.where(fold_train[:, j] == 1,
                                                fold_train[:, j] * upsample_factor,
                                                fold_train[:, j])
                elif neg_count / fold_train.shape[0] < 0.3:
                    fold_train[:, j] = np.where(fold_train[:, j] == 0,
                                                fold_train[:, j] * upsample_factor,
                                                fold_train[:, j])
            fold_train_sparse = csr_matrix(fold_train)
            
            # Validation interactions (untouched).
            fold_val = interactions[val_idx].toarray()
            fold_val_sparse = csr_matrix(fold_val)
            
            # Train LightFM on this fold.
            model_cv = LightFM(loss=loss, no_components=no_components,
                               learning_rate=learning_rate,
                               user_alpha=user_alpha,
                               item_alpha=item_alpha,
                               random_state=42)
            model_cv.fit(fold_train_sparse,
                         user_features=X_train_cv,
                         item_features=item_features,
                         epochs=epochs,
                         num_threads=4)
            
            # Standard precision@3.
            prec = precision_at_k(model_cv, fold_val_sparse,
                                  user_features=X_val_cv,
                                  item_features=item_features,
                                  k=3).mean()
            # Compute custom top-3 accuracy using our function.
            top3_acc = compute_top3_accuracy_for_fold(model_cv, X_val_cv, fold_val_sparse, item_features, k=3)
            
            fold_top3_acc.append(top3_acc)
            fold_prec.append(prec)
        
        avg_top3_acc = np.mean(fold_top3_acc)
        avg_prec = np.mean(fold_prec)
        
        grid_results.append({
            'loss': loss,
            'no_components': no_components,
            'learning_rate': learning_rate,
            'epochs': epochs,
            'user_alpha': user_alpha,
            'item_alpha': item_alpha,
            'top3_accuracy': avg_top3_acc,
            'precision@3': avg_prec
        })
        
        print(f"Params: loss={loss}, components={no_components}, "
              f"lr={learning_rate}, epochs={epochs}, user_alpha={user_alpha}, item_alpha={item_alpha} -> "
              f"Top3 Accuracy: {avg_top3_acc:.4f}, Precision@3: {avg_prec:.4f}")
    
    best_params = max(grid_results, key=lambda x: x['top3_accuracy'])
    return best_params, grid_results

def evaluate_lightfm_model(X_train_full, X_test, Y_train_bin, Y_test_bin, feature_set, best_params, label_cols):
    # 1. Feature Selection and Standardization
    X_train_selected = X_train_full[feature_set].copy()
    X_test_selected = X_test[feature_set].copy()
    
    scaler_final = StandardScaler()
    X_train_final = scaler_final.fit_transform(X_train_selected)
    X_test_final = scaler_final.transform(X_test_selected)
    
    # Convert to sparse matrices for efficiency
    user_features_train_final = csr_matrix(X_train_final)
    user_features_test_final = csr_matrix(X_test_final)

    # 2. Prepare Interaction Data
    final_interactions_train = csr_matrix(Y_train_bin.values)
    final_interactions_test = csr_matrix(Y_test_bin.values)
    num_items = len(label_cols)
    final_item_features = identity(num_items, format='csr')

    # 3. Train Final Model with Best Parameters
    model_final = LightFM(
        loss=best_params['loss'],
        no_components=best_params['no_components'],
        learning_rate=best_params['learning_rate'],
        user_alpha=best_params['user_alpha'],
        item_alpha=best_params['item_alpha'],
        random_state=42
    )

    model_final.fit(
        final_interactions_train,
        user_features=user_features_train_final,
        item_features=final_item_features,
        epochs=best_params['epochs'],
        num_threads=4
    )

    # 4. Evaluation Metrics
    # Precision@3
    final_precision = precision_at_k(
        model_final,
        final_interactions_test,
        user_features=user_features_test_final,
        item_features=final_item_features,
        k=3
    ).mean()

    # Custom Top-3 Accuracy
    custom_top3_accuracy = compute_top3_accuracy_for_fold(
        model_final, 
        user_features_test_final, 
        final_interactions_test, 
        final_item_features, 
        k=3
    )

    # 5. Generate Recommendations (Key Part)
    top3_recommendations = {}
    for user_id in range(user_features_test_final.shape[0]):
        # Predict scores for all items
        scores = model_final.predict(
            user_id, 
            np.arange(num_items),
            user_features=user_features_test_final,
            item_features=final_item_features
        )
        
        # Get indices of top 3 highest scores
        top3_indices = np.argsort(-scores)[:3]
        
        # Map indices to product names
        recommended_products = [label_cols[idx] for idx in top3_indices]
        top3_recommendations[user_id] = recommended_products

    return {
        'precision@3': final_precision,
        'custom_top3_accuracy': custom_top3_accuracy
    }, top3_recommendations

### In our feature engineering process, we first apply Principal Component Analysis (PCA) to the original data to reduce dimensionality and capture the most significant variance in a set of orthogonal components. This initial PCA step compresses the data into a more manageable form and mitigates issues such as multicollinearity. Subsequently, we generate polynomial features based on these PCA-transformed components to capture nonlinear interactions among them. By applying polynomial transformations after PCA, we effectively model complex relationships in the data while avoiding the exponential increase in dimensionality that would occur if polynomial features were generated on the original high-dimensional dataset.

Run Grid Search for PCA where `n_components=22`

In [ ]:
pca_no_poly = PCA(n_components=22, random_state=42)  # Adjust n_components as needed.
X_pca = pca_no_poly.fit_transform(X_expanded_scaled)
pca_feature_names = [f'pca_no_poly_{i}' for i in range(X_pca.shape[1])]

df_pca_no_poly = pd.DataFrame(X_pca, columns=pca_feature_names, index=df_imputed_original.index)

final_feature_set = pca_feature_names

# Split the PCA-transformed data and binarized labels into train and test sets.
X_train_pca, X_test_pca, Y_train_bin, Y_test_bin = train_test_split(df_pca_no_poly, Y_bin, test_size=0.2, random_state=42)

print("\n--- Running grid search for PCA features (no polynomial interactions) ---")
best_params_pca, all_results_pca = grid_search_cv(final_feature_set, X_train_pca, Y_train_bin)
print(f"Best PCA params: {best_params_pca}")


--- Running grid search for PCA features (no polynomial interactions) ---
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=1e-05, item_alpha=1e-05 -> Top3 Accuracy: 0.7891, Precision@3: 0.6537
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=1e-05, item_alpha=0.0001 -> Top3 Accuracy: 0.7891, Precision@3: 0.6537
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=0.0001, item_alpha=1e-05 -> Top3 Accuracy: 0.7891, Precision@3: 0.6537
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=0.0001, item_alpha=0.0001 -> Top3 Accuracy: 0.7895, Precision@3: 0.6537
Params: loss=warp, components=16, lr=0.001, epochs=50, user_alpha=1e-05, item_alpha=1e-05 -> Top3 Accuracy: 0.7888, Precision@3: 0.6539
Params: loss=warp, components=16, lr=0.001, epochs=50, user_alpha=1e-05, item_alpha=0.0001 -> Top3 Accuracy: 0.7888, Precision@3: 0.6539
Params: loss=warp, components=16, lr=0.001, epochs=50, user_alpha=0.0001, item_alpha=1e-05 -> Top3 Accur

In [ ]:
metrics_pca, recommendations_pca = evaluate_lightfm_model(
    X_train_pca, X_test_pca, Y_train_bin, Y_test_bin, final_feature_set, best_params_pca, label_cols
)

print("\nFinal Model Metrics using PCA on expanded data (no polynomial interactions):")
for k, v in metrics_pca.items():
    print(f"{k}: {v:.4f}")

print("\nTop 3 product recommendations for sample test users using PCA (no polynomial interactions):")
for uid in list(recommendations_pca.keys())[:5]:
    print(f"User {uid}: {recommendations_pca[uid]}")


Final Model Metrics using PCA on expanded data (no polynomial interactions):
precision@3: 0.7219
custom_top3_accuracy: 0.8675

Top 3 product recommendations for sample test users using PCA (no polynomial interactions):
User 0: ['Label_Travel_Rewards_Card', 'Label_Rewards_Credit_Card', 'Label_Insurance_Solutions']
User 1: ['Label_Travel_Rewards_Card', 'Label_Home_Improvement_Loan', 'Label_Insurance_Solutions']
User 2: ['Label_Travel_Rewards_Card', 'Label_Digital_Financing', 'Label_Home_Improvement_Loan']
User 3: ['Label_Travel_Rewards_Card', 'Label_Rewards_Credit_Card', 'Label_Commodity_Investment_Services']
User 4: ['Label_Auto_Vehicle_Financing', 'Label_Digital_Financing', 'Label_Home_Improvement_Loan']


## Using PCA does not gurantee better results as compared to the model without PCA, hence we try to use polynomial features

In [ ]:
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_poly = poly.fit_transform(X_expanded)
poly_feature_names = poly.get_feature_names_out(expanded_features)

scaler_poly = StandardScaler()
X_poly_scaled = scaler_poly.fit_transform(X_poly)

n_total = X_poly_scaled.shape[1] 

## Find number of component with 90% cumulative variance explained, in this case, n = 53, but that creates more features than original model. Hence we first experiment with n = 24, where 80% of variance explained, then we proceed with n = 53.

In [ ]:
for n in range(1, n_total + 1):
    pca_polytemp = PCA(n_components=n, random_state=42)
    pca_polytemp.fit(X_poly_scaled)
    cum_polyexplained = np.sum(pca_polytemp.explained_variance_ratio_)
    print(f"n_components = {n}: Cumulative explained variance = {cum_polyexplained:.4f}")

n_components = 1: Cumulative explained variance = 0.1768
n_components = 2: Cumulative explained variance = 0.2424
n_components = 3: Cumulative explained variance = 0.2954
n_components = 4: Cumulative explained variance = 0.3412
n_components = 5: Cumulative explained variance = 0.3818
n_components = 6: Cumulative explained variance = 0.4182
n_components = 7: Cumulative explained variance = 0.4525
n_components = 8: Cumulative explained variance = 0.4845
n_components = 9: Cumulative explained variance = 0.5161
n_components = 10: Cumulative explained variance = 0.5455
n_components = 11: Cumulative explained variance = 0.5736
n_components = 12: Cumulative explained variance = 0.5994
n_components = 13: Cumulative explained variance = 0.6238
n_components = 14: Cumulative explained variance = 0.6479
n_components = 15: Cumulative explained variance = 0.6694
n_components = 16: Cumulative explained variance = 0.6900
n_components = 17: Cumulative explained variance = 0.7096
n_components = 18: Cumu

KeyboardInterrupt: 

In [ ]:
pcapoly = PCA(n_components=24, random_state=42)
X_poly_pca = pcapoly.fit_transform(X_poly_scaled)
pca_feature_names = [f'pca_{i}' for i in range(X_poly_pca.shape[1])]

# Instead of concatenating with the original expanded features, 
# we use only the PCA-transformed polynomial features as the final engineered features.
df_engineered = pd.DataFrame(X_poly_pca, columns=pca_feature_names, index=df_imputed_original.index)
final_feature_set = pca_feature_names

# -----------------------------------------------------------
# Step 5: Split the engineered data and binarized labels into train and test sets.
X_train_eng_full, X_test_eng, Y_train_bin, Y_test_bin = train_test_split(
    df_engineered, Y_bin, test_size=0.2, random_state=42
)

print("\n--- Running grid search for engineered features (using only PCA components) ---")
best_params_eng, all_results_eng = grid_search_cv(final_feature_set, X_train_eng_full, Y_train_bin)
print(f"Best engineered params: {best_params_eng}")


--- Running grid search for engineered features (using only PCA components) ---
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=1e-05, item_alpha=1e-05 -> Top3 Accuracy: 0.7730, Precision@3: 0.6456
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=1e-05, item_alpha=0.0001 -> Top3 Accuracy: 0.7730, Precision@3: 0.6456
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=0.0001, item_alpha=1e-05 -> Top3 Accuracy: 0.7730, Precision@3: 0.6456
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=0.0001, item_alpha=0.0001 -> Top3 Accuracy: 0.7730, Precision@3: 0.6456
Params: loss=warp, components=16, lr=0.001, epochs=50, user_alpha=1e-05, item_alpha=1e-05 -> Top3 Accuracy: 0.7683, Precision@3: 0.6419
Params: loss=warp, components=16, lr=0.001, epochs=50, user_alpha=1e-05, item_alpha=0.0001 -> Top3 Accuracy: 0.7672, Precision@3: 0.6412
Params: loss=warp, components=16, lr=0.001, epochs=50, user_alpha=0.0001, item_alpha=1e-05 -> Top3

In [ ]:
# Evaluate final model using the engineered feature set.
metrics_eng, recommendations_eng = evaluate_lightfm_model(
    X_train_eng_full, X_test_eng, Y_train_bin, Y_test_bin, final_feature_set, best_params_eng, label_cols
)

print("\nFinal Engineered Model Metrics:")
for metric, value in metrics_eng.items():
    print(f"{metric}: {value:.4f}")

print("\nTop 3 product recommendations for sample test users (Engineered Features):")
for uid in list(recommendations_eng.keys())[:5]:
    print(f"User {uid}: {recommendations_eng[uid]}")


Final Engineered Model Metrics:
precision@3: 0.7179
custom_top3_accuracy: 0.8599

Top 3 product recommendations for sample test users (Engineered Features):
User 0: ['Label_Travel_Rewards_Card', 'Label_Rewards_Credit_Card', 'Label_Insurance_Solutions']
User 1: ['Label_Travel_Rewards_Card', 'Label_Rewards_Credit_Card', 'Label_Home_Improvement_Loan']
User 2: ['Label_Travel_Rewards_Card', 'Label_Insurance_Solutions', 'Label_Digital_Financing']
User 3: ['Label_Travel_Rewards_Card', 'Label_Rewards_Credit_Card', 'Label_Commodity_Investment_Services']
User 4: ['Label_Auto_Vehicle_Financing', 'Label_Digital_Financing', 'Label_Travel_Rewards_Card']


In [ ]:
pcapoly = PCA(n_components=53, random_state=42)
X_poly_pca = pcapoly.fit_transform(X_poly_scaled)
pca_feature_names = [f'pca_{i}' for i in range(X_poly_pca.shape[1])]

# Instead of concatenating with the original expanded features, 
# we use only the PCA-transformed polynomial features as the final engineered features.
df_engineered = pd.DataFrame(X_poly_pca, columns=pca_feature_names, index=df_imputed_original.index)
final_feature_set = pca_feature_names

# -----------------------------------------------------------
# Step 5: Split the engineered data and binarized labels into train and test sets.
X_train_eng_full, X_test_eng, Y_train_bin, Y_test_bin = train_test_split(
    df_engineered, Y_bin, test_size=0.2, random_state=42
)

print("\n--- Running grid search for engineered features (using only PCA components) ---")
best_params_eng, all_results_eng = grid_search_cv(final_feature_set, X_train_eng_full, Y_train_bin)
print(f"Best engineered params: {best_params_eng}")


--- Running grid search for engineered features (using only PCA components) ---
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=1e-05, item_alpha=1e-05 -> Top3 Accuracy: 0.7983, Precision@3: 0.6586
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=1e-05, item_alpha=0.0001 -> Top3 Accuracy: 0.7983, Precision@3: 0.6586
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=0.0001, item_alpha=1e-05 -> Top3 Accuracy: 0.7979, Precision@3: 0.6583
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=0.0001, item_alpha=0.0001 -> Top3 Accuracy: 0.7983, Precision@3: 0.6586
Params: loss=warp, components=16, lr=0.001, epochs=50, user_alpha=1e-05, item_alpha=1e-05 -> Top3 Accuracy: 0.7970, Precision@3: 0.6573
Params: loss=warp, components=16, lr=0.001, epochs=50, user_alpha=1e-05, item_alpha=0.0001 -> Top3 Accuracy: 0.7971, Precision@3: 0.6571
Params: loss=warp, components=16, lr=0.001, epochs=50, user_alpha=0.0001, item_alpha=1e-05 -> Top3

In [ ]:
# Evaluate final model using the engineered feature set.
metrics_eng, recommendations_eng = evaluate_lightfm_model(
    X_train_eng_full, X_test_eng, Y_train_bin, Y_test_bin, final_feature_set, best_params_eng, label_cols
)

print("\nFinal Engineered Model Metrics:")
for metric, value in metrics_eng.items():
    print(f"{metric}: {value:.4f}")

print("\nTop 3 product recommendations for sample test users (Engineered Features):")
for uid in list(recommendations_eng.keys())[:5]:
    print(f"User {uid}: {recommendations_eng[uid]}")


Final Engineered Model Metrics:
precision@3: 0.7329
custom_top3_accuracy: 0.8850

Top 3 product recommendations for sample test users (Engineered Features):
User 0: ['Label_Travel_Rewards_Card', 'Label_Rewards_Credit_Card', 'Label_Insurance_Solutions']
User 1: ['Label_Travel_Rewards_Card', 'Label_Rewards_Credit_Card', 'Label_Insurance_Solutions']
User 2: ['Label_Rewards_Credit_Card', 'Label_Travel_Rewards_Card', 'Label_Digital_Financing']
User 3: ['Label_Travel_Rewards_Card', 'Label_Commodity_Investment_Services', 'Label_Rewards_Credit_Card']
User 4: ['Label_Auto_Vehicle_Financing', 'Label_Digital_Financing', 'Label_Home_Improvement_Loan']


## Conclusion:
Using polynomial features has proven to yield significantly improved results in our modeling efforts by uncovering complex, nonlinear interactions that are otherwise hidden in the original data. After applying PCA to distill the most important and uncorrelated components from our dataset, we generate polynomial features on these components to effectively capture higher-order relationships among them. This approach allows us to model intricate dependencies without suffering from the curse of dimensionality that would occur if polynomial expansion were applied to the original, high-dimensional data. Empirical evidence from our experiments shows that models incorporating these polynomial features consistently outperform those using only linear combinations of the principal components, indicating that the nonlinear interactions captured are indeed critical to the underlying data structure. In conclusion, the use of polynomial features, built upon a PCA-transformed dataset, not only enhances predictive accuracy but also provides a more nuanced understanding of the complex relationships driving the behavior of our data.